In [1]:
from pytorch_model_summary import summary

In [2]:
import torch

In [3]:
from model import import_model
from argparse import Namespace
opt = Namespace()

opt.config = {
    'model': {
        'name': 'sye',  # or 'mobileie', 'retinex', 'retinex_v2', 'sye'
        'type': 'original',  # or 're-parameterized'
        'channels': 12,
        'rep_scale': 4,
        'pretrained': None,
        'pretrained_date': None,
        'need_slim': False,
    }
}
opt.model_task = 'isp'
opt.device = 'cpu'

retinex_model = import_model(opt)


In [4]:
print(summary(retinex_model, torch.zeros((1, 4, 512, 512)), show_input=False))

------------------------------------------------------------------------------------
                Layer (type)           Output Shape         Param #     Tr. Param #
   QuadraticConnectionUnit-1      [1, 12, 512, 512]          68,796          68,796
   QuadraticConnectionUnit-2      [1, 12, 512, 512]          26,340          26,340
         AdaptiveAvgPool2d-3          [1, 12, 1, 1]               0               0
              ConvRepPoint-4          [1, 12, 1, 1]           2,508           2,508
                     PReLU-5          [1, 12, 1, 1]              12              12
              ConvRepPoint-6          [1, 12, 1, 1]           2,508           2,508
                   Sigmoid-7          [1, 12, 1, 1]               0               0
              PixelShuffle-8     [1, 3, 1024, 1024]               0               0
                  ConvRep3-9     [1, 3, 1024, 1024]           1,635           1,635
Total params: 101,799
Trainable params: 101,799
Non-trainable params: 0
---

In [5]:
slim_model = retinex_model.slim()

In [6]:
print(summary(slim_model, torch.zeros((1, 4, 512, 512)), show_input=False))

-------------------------------------------------------------------------------------
                 Layer (type)           Output Shape         Param #     Tr. Param #
   QuadraticConnectionUnitS-1      [1, 12, 512, 512]           3,756           3,756
   QuadraticConnectionUnitS-2      [1, 12, 512, 512]           1,476           1,476
          AdaptiveAvgPool2d-3          [1, 12, 1, 1]               0               0
                     Conv2d-4          [1, 12, 1, 1]             156             156
                      PReLU-5          [1, 12, 1, 1]              12              12
                     Conv2d-6          [1, 12, 1, 1]             156             156
                    Sigmoid-7          [1, 12, 1, 1]               0               0
               PixelShuffle-8     [1, 3, 1024, 1024]               0               0
                     Conv2d-9     [1, 3, 1024, 1024]              84              84
Total params: 5,640
Trainable params: 5,640
Non-trainable params

In [7]:
retinex_model.eval()
slim_model.eval();

In [7]:
input = torch.randn(1, 4, 448, 448)

# Write a time checking by inferences 20 times
import time 

start_time = time.time()
for _ in range(20):
    output1 = slim_model(input)
end_time = time.time()
print(f"Retinex model average inference time: {(end_time - start_time) / 20:.6f} seconds")

Retinex model average inference time: 0.404707 seconds
